In [1]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pickle

# Load the numpy arrays
X_train = np.load('D:/JATHURSH/Jupyter_notebook_projects/Saved_stuff_from_drowsiness_detection_project/full_data_train_test_validation_split/X_train.npy')

X_test = np.load('D:/JATHURSH/Jupyter_notebook_projects/Saved_stuff_from_drowsiness_detection_project/full_data_train_test_validation_split/X_test.npy')

X_val = np.load('D:/JATHURSH/Jupyter_notebook_projects/Saved_stuff_from_drowsiness_detection_project/full_data_train_test_validation_split/X_val.npy')

y_train = np.load('D:/JATHURSH/Jupyter_notebook_projects/Saved_stuff_from_drowsiness_detection_project/full_data_train_test_validation_split/y_train.npy')

y_test = np.load('D:/JATHURSH/Jupyter_notebook_projects/Saved_stuff_from_drowsiness_detection_project/full_data_train_test_validation_split/y_test.npy')

y_val = np.load('D:/JATHURSH/Jupyter_notebook_projects/Saved_stuff_from_drowsiness_detection_project/full_data_train_test_validation_split/y_val.npy')

In [2]:
# preprocessing
# preprocess using mobilNetV2 - preprocess_input method
X_train_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(X_train)

In [4]:
X_val_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(X_val)
X_test_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(X_test)    

In [5]:
# free up mem
del X_train
del X_test
del X_val

In [6]:
# pretrained model - MobileNetV2
# all should be numpy arrays or tensors to give to MobileNetV2
"""
tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=None,                 # Shape of input data (MobileNets support any input size greater than 32 x 32, with larger image sizes offering better performance.) 
    alpha=1.0,                        # Width multiplier for controlling network width (default is 1.0)
    include_top=True,                 # Include fully connected top layers (default is True)
    weights='imagenet',               # Weight initialization ('imagenet' for pre-trained weights, None for random initialization)
    input_tensor=None,                # Optional input tensor (default is None)
    pooling=None,                     # Pooling type for the last layer ('avg', 'max', or None; default is None)
    classes=1000,                     # Number of output classes (default is 1000 for ImageNet)
    classifier_activation='softmax',   # Activation function for output layer (default is 'softmax')
    **kwargs                          # Additional keyword arguments
)
"""
# we dont have to set anything and keep its default values

MobileNetV2_model = tf.keras.applications.mobilenet_v2.MobileNetV2()

# Make all layers in the model non-trainable - freeze
for layer in MobileNetV2_model.layers:
    layer.trainable = False

In [7]:
# Get the base model's input and output
base_input = MobileNetV2_model.layers[0].input
base_output = MobileNetV2_model.layers[-3].output

# custom dense layers
x = tf.keras.layers.GlobalAveragePooling2D()(base_output)
x = tf.keras.layers.Dense(256, activation='relu')(x)  # Add a dense layer with 256 units and ReLU activation
x = tf.keras.layers.Dropout(0.5)(x)  # Add dropout for regularization
x = tf.keras.layers.Dense(128, activation='relu')(x)  # Add another dense layer with 128 units and ReLU activation

# Final output layer
final_output = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation

In [8]:
drowsiness_detection_model_2 = tf.keras.Model(
    inputs=base_input,
    outputs=final_output
)

In [10]:
# Compile and train the model using X_train_preprocessed as input
drowsiness_detection_model_2.compile(optimizer='adam', 
                                   loss='binary_crossentropy', 
                                   metrics=['accuracy'])
drowsiness_detection_model_2.fit(X_train_preprocessed, 
                               y_train, 
                               epochs=10, 
                               batch_size=4, 
                               validation_data=(X_val_preprocessed, y_val))

MemoryError: Unable to allocate 33.3 GiB for an array with shape (59428, 224, 224, 3) and data type float32